In [16]:
import sys
import fileinput
import json
import pprint
import nltk
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

In [2]:
# https://docs.python.org/3.6/library/fileinput.html
def read_file():
  with fileinput.input(files=('Tweets2.txt')) as f:
    for line in f:
      tweet = json.loads(line)
      tweets.append(tweet)


In [23]:
def extract_geo():
  for tweet in tweets:
    if(tweet['geo'] is not None):
      t = {
        "coordinates": tweet['coordinates']['coordinates'],
        "date": tweet['created_at'],
        "text": tweet['text']
        }
      geo_tweets.append(t)
      

In [24]:
x = [g['coordinates'][0] for g in geo_tweets]
xmin, xmax = min(x), max(x)
print(xmin)
print(xmax)

y = [g['coordinates'][1] for g in geo_tweets]
ymin, ymax = min(y), max(y)
print(ymin)
print(ymax)

144.44419
145.5317846
-38.4712149
-37.330222


In [5]:
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize_text(text):
  sentences = nltk.sent_tokenize(text)
  word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
  return word_tokens


def remove_stopwords(tokens):
  stopword_list = nltk.corpus.stopwords.words('english')
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  return filtered_tokens



In [7]:
tweets = []
geo_tweets = []
  
read_file()
extract_geo()


144.44419
145.5317846
-38.4712149
-37.330222
